In [1]:
import requests
import bs4 
import pandas

In [2]:
def get_directories(passed_html):
    """
    Passed a GitHub-Repo this function finds all of the child directories under that repo and returns a list of urls.
    """
    sub_directories = passed_html.find_all("a", class_="js-navigation-open Link--primary") # Finding all of the internal navigation links

    repositories = []   # A list to store the new repository urls.
    for sub_directory in sub_directories:   # Looping through the child directories as strings 
        for js_string in str(sub_directory).split('"'): # Retrieving the java-script strings 
            if js_string.startswith("/"):   # Finding the internal links by the first character 
                repositories.append("".join(["https://github.com", js_string])) # Adding the appropriate preface to the url.

    return repositories # Returning the list of internal directories

In [3]:
http_request = requests.get("https://github.com/SharmaLlama/ticktack/tree/main/src/data/datasets") # Getting the root directory
html_parser = bs4.BeautifulSoup(http_request.text, "html.parser")   # Passing the raw text of the directy as html for better manipulation 

In [4]:
data_sets = []  # Initialising a list to store the directories
for dir in get_directories(html_parser):    # Finding all the child directories of the root directory 
    data_set = requests.get(dir)    # Getting the raw http response from the child directory 
    passed_data = bs4.BeautifulSoup(data_set.text, "html.parser")   # Passing the child directory as html 

    data_sets.append(get_directories(passed_data))  # Storing the children's children in the data_sets list

In [5]:
for i, directory_data_sets in enumerate(data_sets): # Looping over the data sets
    for data_set_url in directory_data_sets:    # getting the urls of the data_sets
        if data_set_url.find("csv") == -1:  # Checking that the leaves of the directory have been reached 
            data_set = requests.get(data_set_url)   # Getting a response from the data set repo
            passed_data = bs4.BeautifulSoup(data_set.text, "html.parser")   # Passing the data set as
            data_sets[i] = get_directories(passed_data) # Updating the stored entry to make the list homogeneous

In [21]:
data = []   # List to store the python data sets

for data_urls in data_sets: # Loopig through the data sets in found by the program 
    for data_url in data_urls:  # Extracting the individual urls 
        data_set = requests.get(data_url)   # getting the http response for the data
        data_set = bs4.BeautifulSoup(data_set.text, "html.parser")  # Passing the data html 

        datum = {    # Storing the data as a python dictionary 
            "year": [], # A column to store the year 
            "d14c": [], # A column to store the carbon 14 concentrations 
            "sig_d14c": []  # A columns to store the error in the carbon 14 concentrations 
        }
        
        tabular_rows = data_set.find_all("tr") # retrieving the tabular data 
        columns_names = ["year", "d14c", "sig_d14c"]

        for tabular_row in tabular_rows:  # Looping through each line of the table 
            column_counter = 0  # Keeping track of the current column in the data 
            tabular_columns = tabular_row.find_all("td")    # Finding all of the entries in the row

            for tabular_column in tabular_columns:  # looping over the columns 
                if tabular_column.has_attr("id"):   # Checking if the searcher is in a data column 
                    column_counter = 0  # Resetting the column indicating new row
                else:
                    tabular_entry = str(tabular_column).strip("<td>").strip("</td>")   # Retrieving data
                    datum[columns_names[column_counter]].append(tabular_entry)    # Adding the data to storage 
                    column_counter += 1 # Moving along one column
                
        data.append(datum)

        # pandas.DataFrame(datum).to_csv()


Not all of the data sets, are lining up. I need to improve this with a conditional statement and then get the data stored.

In [22]:
data

[{'year': '-5246', 'd14c': '98.94', 'sig_d14c': '2.26'},
 {'year': '-5247', 'd14c': '97.46', 'sig_d14c': '1.71'},
 {'year': '-5247', 'd14c': '96.62', 'sig_d14c': '1.77'},
 {'year': '-5251', 'd14c': '99.6', 'sig_d14c': '1.87'},
 {'year': '-5396', 'd14c': '86.9', 'sig_d14c': '3.4'},
 {'year': '-5400', 'd14c': '91.7', 'sig_d14c': '3'},
 {'year': '-5381', 'd14c': '84.9', 'sig_d14c': '2.3'},
 {'year': '-633', 'd14c': '-0.3', 'sig_d14c': '2.4'},
 {'year': '-633', 'd14c': '2.4', 'sig_d14c': '2.3'},
 {'year': '-7147', 'd14c': '82.91', 'sig_d14c': '2.95'},
 {'year': '-7147', 'd14c': '83.17', 'sig_d14c': '1.92'},
 {'year': '-7150', 'd14c': '84.33', 'sig_d14c': '2.47'},
 {'year': '780', 'd14c': '-8.8', 'sig_d14c': '1.4'},
 {'year': '780', 'd14c': '-12.9', 'sig_d14c': '1.6'},
 {'year': '780', 'd14c': '-12.1', 'sig_d14c': '1.6'},
 {'year': '780', 'd14c': '-11.6', 'sig_d14c': '2.3'},
 {'year': '780', 'd14c': '-13.8', 'sig_d14c': '2.3'},
 {'year': '780', 'd14c': '-12.4', 'sig_d14c': '2.3'},
 {'year':